# Join the different tables

In [73]:
import pandas as pd

product = pd.read_csv('filtered_products.csv')
internetsales = pd.read_csv('factinternetsales.csv')
customer = pd.read_csv('dimcustomer.csv')

## Merge the datasets

In [74]:
merged_df = pd.merge(internetsales, product[['productkey', 'englishproductname', 'standardcost', 'dealerprice', 'status']], on='productkey')

In [75]:
merged_df = pd.merge(merged_df,customer, on='customerkey')

## Select columns

In [76]:
customer_cols=['customerkey', 'geographykey', 'birthdate', 'maritalstatus', 'gender', 'totalchildren', 'yearlyincome']
sales_cols = ['productkey', 'unitprice', 'extendedamount', 'totalproductcost', 'freight', 'orderdate', 'duedate']
product_cols=['englishproductname', 'status']

flt_cols = sales_cols + product_cols + customer_cols

In [77]:
final_df = merged_df[flt_cols]

## Prepare date cols

In [78]:
date_cols = ['orderdate', 'duedate', 'birthdate']

for col in date_cols:
    final_df[col] = pd.to_datetime(final_df[col], format='%Y-%m-%d')

/tmp/ipykernel_612/1131269319.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[col] = pd.to_datetime(final_df[col], format='%Y-%m-%d')
/tmp/ipykernel_612/1131269319.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[col] = pd.to_datetime(final_df[col], format='%Y-%m-%d')
/tmp/ipykernel_612/1131269319.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [79]:
final_df.to_csv('merged_df.csv', index=False)

## Calculate the dataset Metadata

In [80]:
from ydata.dataset import Dataset
from ydata.metadata import Metadata

dataset = Dataset(final_df)
m = Metadata(dataset)

[########################################] | 100% Completed | 102.61 ms
[########################################] | 100% Completed | 102.61 ms
[########################################] | 100% Completed | 1.29 sms
[########################################] | 100% Completed | 4.47 sms


In [81]:
m.update_datatypes({'productkey':'categorical', 
                    'englishproductname': 'categorical', 
                    'customerkey': 'categorical'})

In [82]:
m.save('metadata_merged.pkl')

## Profile the dataset

In [85]:
from ydata.profiling import ProfileReport

report = ProfileReport(final_df, title='Internet sales per customer and product')
report_html = report.to_html()

/home/ydata/.venv/lib/python3.10/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Pipeline outputs

In [84]:
##add here the outputs logic
import json

profile_pipeline_output = {
    'outputs' : [
        {
            'type': 'table',
            'storage': 'inline',
            'format': 'csv',
            'header': list(final_df.columns),
            'source': final_df.sample(10).to_csv(header=False, index=True)
        },
        {
          'type': 'web-app',
          'storage': 'inline',
          'source': report_html,
        },
    ]
  }

with open('mlpipeline-ui-metadata.json', 'w') as metadata_file:
    json.dump(profile_pipeline_output, metadata_file)